Kaggleのタイタニックタスク  
https://www.kaggle.com/c/titanic/

k_yoshida 2018/04/19

In [0]:
import pandas as pd
import io
import requests
import xgboost as xgb
import numpy as np
from sklearn import preprocessing
from google.colab import files

In [0]:
url="https://raw.githubusercontent.com/kuiski/kaggle_titanic/master/input/train.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [135]:
c[:3]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [136]:
feature_data = c.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
feature_data[:4]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S


In [137]:
label_data = c.loc[:, ['Survived']]
label_data[:3]

,Survived
0,0
1,1
2,1


In [0]:
sex_encorder = preprocessing.LabelEncoder()
sex_encorder.fit(c['Sex'].astype(str))
feature_data['Sex'] = sex_encorder.transform(feature_data['Sex'])

In [0]:
embarked_encorder = preprocessing.LabelEncoder()
embarked_encorder.fit(c['Embarked'].astype(str))
feature_data['Embarked'] = embarked_encorder.transform(feature_data['Embarked'].astype(str))

In [140]:
feature_data.loc[:3]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2


In [141]:
xgb_model = xgb.XGBClassifier(colsample_bytree=0.9, max_depth=7, subsample=0.81)
xgb_model.fit(feature_data, label_data)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.81)

In [0]:
url="https://raw.githubusercontent.com/kuiski/kaggle_titanic/master/input/test.csv"
s=requests.get(url).content
test=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [143]:
test[:3]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [144]:
test_data = test.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_data[:3]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q


In [145]:
test_data['Embarked'] = embarked_encorder.transform(test_data['Embarked'].astype(str))
test_data['Sex'] = sex_encorder.transform(test_data['Sex'])
test_data[:3]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1


In [146]:
predictions = xgb_model.predict(test_data)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [147]:
predictions[:3]

array([0, 0, 0])

In [148]:
outputs = test.loc[:, ['PassengerId']]
outputs['Survived'] = predictions
outputs[:3]

,PassengerId,Survived
0,892,0
1,893,0
2,894,0


In [0]:
filename = "output.csv"
outputs.to_csv(filename, index=False)
files.download(filename)